In [1]:
!pip install transformers

In [2]:
!pip install SentencePiece

In [3]:
!pip install accelerate

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00




---





---



In [6]:
from transformers import pipeline, set_seed

In [7]:
en_de_translator = pipeline("translation_en_to_de", model="t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
set_seed(1)

## WMT2014en-de

In [9]:
with open('/content/newstest2014.de.txt', 'r', encoding='utf-8') as d:
    de = d.read()
with open('/content/newstest2014.en.txt', 'r', encoding='utf-8') as e:
    en = e.read()

In [10]:
de= de.split('\n')
en= en.split('\n')

In [11]:
len(de)==len(en)

True

In [12]:
print(de[0], '\n', en[0])

Orlando Bloom und Miranda Kerr lieben sich noch immer 
 Orlando Bloom and Miranda Kerr still love each other


## BLEU 측정

In [13]:
from nltk.translate import bleu

In [14]:
#warning 무시하기

import warnings

# To mute the specific warning:

#warnings.filterwarnings("ignore", category=UserWarning, module="nltk.translate.bleu_score")

In [15]:
max_tokens= 100
#from warning:
#Your input_length: 71 is bigger than 0.9 * max_length: 75... ///  e.g. translator('...', max_length=400)

In [38]:
cnds= []
bleus= []

for i in range(100):
  result= en_de_translator("translate English to German: "+en[i], max_length=max_tokens)

  cnd= result[0]['translation_text']
  ref= de[i]
  cnds.append(cnd)
  #print(cnd, ref)

  cnd= cnd.split()
  ref= ref.split()

  score= bleu([ref], cnd, weights=(0.25, 0.25, 0.25, 0.25))
  bleus.append(score)

  if i>0 and i%10==0:
    print("now, the bleu is ", sum(bleus)/i)

now, the bleu is  0.28576254529713385
now, the bleu is  0.19948631359359043


Your input_length: 96 is bigger than 0.9 * max_length: 100. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


now, the bleu is  0.1893824437351826
now, the bleu is  0.1654943630195316
now, the bleu is  0.15101667388933643
now, the bleu is  0.1444593093038912
now, the bleu is  0.15318404586084777
now, the bleu is  0.14501504410698113
now, the bleu is  0.13394266413575384


Your input_length: 98 is bigger than 0.9 * max_length: 100. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


t5의 번역은 "I am good." 이런식인데 reference 문장은 "I am good ." 이런 식

문장부호 앞 공백 때문에 점수 낮아지는 걸수도 있어서, 라이브러리 data를 사용해봅니다

(같은 건 줄 알았는데 row 수도 다르네 그냥 다른 데이터인가?)

## 테스트2



In [23]:
from datasets import load_dataset

dataset = load_dataset("wmt14", "de-en")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [24]:
test_dataset = dataset["test"]

In [25]:
test_dataset

Dataset({
    features: ['translation'],
    num_rows: 3003
})

In [27]:
test= test_dataset['translation']

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

# Your translations, for example:
translations = ["Your translated sentence 1", "Your translated sentence 2", ...]

# Reference translations from the test dataset
reference_translations = test_dataset["translation"]

# Create a list of reference translations as lists of tokens
references = [[ref.split()] for ref in reference_translations]

# Tokenize your translations
translations = [translation.split() for translation in translations]

# Calculate the BLEU score
bleu_score = corpus_bleu(references, translations)

print("BLEU Score:", bleu_score)


In [29]:
test[0]

{'de': 'Gutach: Noch mehr Sicherheit für Fußgänger',
 'en': 'Gutach: Increased safety for pedestrians'}

In [34]:
cnds= []
bleus= []

for i in range(10):
  result= en_de_translator("translate English to German: "+test[i]['en'], max_length=max_tokens)

  cnd= result[0]['translation_text']
  ref= test[i]['de']
  cnds.append(cnd)
  #print(cnd, ref)

  cnd= cnd.split()
  ref= ref.split()

  score= bleu([ref], cnd, weights=(0.25, 0.25, 0.25, 0.25))
  bleus.append(score)
  print(score)

  # if i>0 and i%30==0:
  #   print("now, the bleu is ", sum(bleus)/i)

6.042433724643145e-78
0.13733894353973464
1.3165594234639305e-231
1.2627076138080564e-231
4.83404070852666e-155
5.767166022187344e-155
0.195647514979229
0.2880209091216406
1.2508498911928379e-231
0.38260294162784475


시상에.. 더 안 좋아졌잖아



---



## t5 결과물 확인:

t5 모델이 번역을 그렇게 못하는가 하면 그것은 아닙니다

In [35]:
test[5]['en']

'The Kluser lights protect cyclists, as well as those travelling by bus and the residents of Bergle.'

In [36]:
test[5]['de']

'Die Kluser-Ampel sichere sowohl Radfahrer als auch Busfahrgäste und die Bergle-Bewohner.'

In [37]:
en_de_translator("translate English to German: "+test[5]['en'], max_length=max_tokens)

[{'translation_text': 'Die Kluser-Lampen schützen Radfahrer, Busfahrer und die Einwohner von Bergle.'}]

n-gram 겹치는게 별로 없어서 같은데.. 여튼 이 추세면 bleu 30점 절대 불가능



---



## 결론

BLEU 측정은 쉬운데, reproduce가 안되고 일차적 원인은 데이터에 있어보임

그리고 datasets보다 txt파일이 더 가능성 있어보임.




대책

1. 임의로 reference의 문장부호 앞 공백을 제거하거나
candidate에 문장 부호 앞에 공백을 추가해도 되는지? (조작일까..?)

2. 초반은 점수 잘 나오는 거 확인 했는데, 평균 bleu가 계속 낮아지는 걸 봐서 중간부터 엄청 낮은 점수 받는걸수도
